In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[5]").appName("VarunSpark").getOrCreate()

25/06/18 05:31:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/18 05:31:12 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/18 05:31:12 WARN Utils: Serv

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
impressionSchema = StructType([\
                                StructField('CreatedTime', TimestampType(),True),\
                                StructField('Type', StringType(), True),\
                                StructField('Amount', IntegerType(), True),\
                                StructField('BrokerCode', StringType(),True)])

In [5]:
df = spark.readStream.format('socket').option('host','localhost').option('port',7766).load()

25/06/18 05:31:14 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
data=  df.select(from_json(col('value').cast('string'),impressionSchema).alias('value'))
data=data.select('value.*')
data.printSchema()

root
 |-- CreatedTime: timestamp (nullable = true)
 |-- Type: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- BrokerCode: string (nullable = true)



In [8]:
res = data.groupby(window(data.CreatedTime,"5 minutes"), data.Type).sum("Amount")

In [9]:
res1 = res.withColumnRenamed("sum(Amount)", "sumdata")

In [10]:
res1 = res1.select('window.start', 'window.end', 'Type', 'sumdata')

In [ ]:
res1.writeStream.format('console').outputMode('update').start().awaitTermination()

25/06/18 05:31:20 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-66b453d8-3a8e-4891-bcf2-5d4b3449edfd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/18 05:32:46 WARN DataStreamer: Slow waitForAckedSeqno took 60174ms (threshold=30000ms). File being written: /tmp/temporary-66b453d8-3a8e-4891-bcf2-5d4b3449edfd/state/0/87/.1.delta.4fda800e-a06e-43c0-8884-45d3cd2b1d54.TID92.tmp, block: BP-1364978691-172.31.64.10-1677754292516:blk_1079491731_5754528, Write pipeline datanodes: [DatanodeInfoWithStorage[172.31.64.12:9866,DS-c7b06f49-e29e-4fd7-8923-882da8ea1f7c,DISK], DatanodeInfoWithStorage[172.31.64.10:9866,DS-c5d3f703-fc05-45d5-9749-75d35b7c1674,DISK], DatanodeInfoWithStorage[172.31.64.16:9866,DS-a14784d9-461c-4107-8dc3-0f461fee974b,DISK]].
25/06/18 05:3

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---+----+-------+
|start|end|Type|sumdata|
+-----+---+----+-------+
+-----+---+----+-------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+---+----+-------+
|start|end|Type|sumdata|
+-----+---+----+-------+
+-----+---+----+-------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+---+----+-------+
|start|end|Type|sumdata|
+-----+---+----+-------+
+-----+---+----+-------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+---+----+-------+
|start|end|Type|sumdata|
+-----+---+----+-------+
+-----+---+----+-------+



25/06/18 11:12:13 WARN TextSocketMicroBatchStream: Stream closed by localhost:7766
